In [2]:
def read_instance(instance_name):
    with open('instance-set/'+ instance_name +'.txt', 'r') as file:
        lines = file.readlines()
    items_count = int(lines[0].strip())
    bin_size = int(lines[1].strip())
    items_dict = {i+1: int(line.strip()) for i, line in enumerate(lines[2:])}
    if items_count != len(items_dict):
        raise ValueError(f"Items count {items_count} does not match the number of items {len(items_dict)}.")
    return items_dict, bin_size

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import math 
from collections import Counter
class Chromosome:
    def __init__(self, value, items_dict, bin_size):
        self.value = value
        self.items_dict = items_dict
        self.bin_size = bin_size
        self.bins = self.construct_bins()
        self.fitness = self.objective_function()

    def construct_bins(self):
        bins = []
        bin_contents = []
        current_bin = 0

        for item_id in self.value:
            item_size = self.items_dict[item_id]
            if not bins or bins[current_bin]['used'] + item_size > self.bin_size:
                bins.append({'used': item_size, 'items': [(item_id, item_size)]})
                current_bin = len(bins) - 1
            else:
                bins[current_bin]['used'] += item_size
                bins[current_bin]['items'].append((item_id, item_size))

        self.bin_contents = [bin['items'] for bin in bins]
        return bins

    def objective_function(self, z=2):
        N = len(self.bins)
        if N == 0:
            return float('inf')
        sum_filled_ratios = sum((bin['used'] / self.bin_size) ** z for bin in self.bins)
        return 1 - (sum_filled_ratios / N)

    def mutate(self, pm=0.8):
        if random.random() < pm:
            position1, position2 = random.sample(range(len(self.value)), 2)
            self.value[position1], self.value[position2] = self.value[position2], self.value[position1]
        return self
    
    def visualize_bins(self, folder_path="Plots"):
        bins_per_row = 6
        num_bins = len(self.bins)
        num_rows = math.ceil(num_bins / bins_per_row)
        color_map = plt.get_cmap('tab20')
        plt.figure(figsize=(15, max(2, num_rows) * 2))

        for i, bin_content in enumerate(self.bin_contents):
            plt.subplot(num_rows, bins_per_row, i + 1)
            y_offset = 0
            for item_index, (item_id, item_size) in enumerate(bin_content):
                color = color_map(item_index % 20)
                plt.bar([0.5], [item_size], bottom=[y_offset], width=0.9, edgecolor='black', color=color)
                text_pos_y = y_offset + item_size / 2
                text = f"id: {item_id} - size: {item_size}"
                plt.text(0.5, text_pos_y, text, ha='center', va='center', fontsize=8, color='white')
                y_offset += item_size

            plt.ylim(0, self.bin_size)
            plt.title(f"Bin {i}")
            plt.xticks([])
            plt.yticks([])

        plt.tight_layout()
        plt.savefig(f"{folder_path}/"+instance_name+".png")
        plt.close()
        
        
    def write_output(self, file_path):
        try:
            with open(file_path, 'w') as outputFile:
                outputFile.write(f"{len(self.bins)}\n")
                for bin in self.bins:
                    items = bin['items']
                    outputFile.write(" ".join(f"{item_id}" for item_id, _ in items) + "\n")
        except Exception as e:
            print(f"An error occurred: {e}")
            
    @staticmethod
    def from_file(file_path, items_dict, bin_size):
        try:
            with open(f"Outputs/output_{file_path}.txt", 'r') as inputFile:
                inputFile.readline()
                item_ids = [int(item_id) for line in inputFile for item_id in line.strip().split()]
                return Chromosome(item_ids, items_dict, bin_size)
        except Exception as e:
            print(f"An error occurred while reading the file: {e}")
            return None
    
    def check_feasibility(self):
        item_counts = Counter()
        for bin in self.bins:
            total_size = 0
            for item_id, item_size in bin['items']:
                total_size += item_size
                item_counts[item_id] += 1

            if total_size > self.bin_size:
                print(f"Bin size exceeded: {total_size} > {self.bin_size}")
                return False

        expected_ids = set(self.items_dict.keys())
        observed_ids = set(item_counts.keys())

        # Check for missing IDs
        if observed_ids != expected_ids:
            missing_ids = expected_ids - observed_ids
            extra_ids = observed_ids - expected_ids
            if missing_ids:
                print(f"Missing item IDs: {missing_ids}")
            if extra_ids:
                print(f"Extra item IDs: {extra_ids}")
            return False

        # Check for duplicates by finding any item_id with a count greater than 1
        duplicates = {item_id for item_id, count in item_counts.items() if count > 1}
        if duplicates:
            print(f"Duplicate item IDs: {duplicates}")
            return False

        return True

In [4]:
import random
def initialize_population(items_count, pop_num):
    population = []
    for _ in range(pop_num):
        value = list(range(1, items_count + 1))
        random.shuffle(value)        
        chromosome = Chromosome(value, items_dict, bin_size)
        population.append(chromosome)
    return population

In [5]:
def histogram_of_bins(population):
    num_bins = [len(chromosome.bins) for chromosome in population]
    plt.figure(figsize=(6,4))
    plt.hist(num_bins, bins=max(num_bins) - min(num_bins), color='skyblue', edgecolor='black')
    plt.title('Histogram of Number of Bins Used in Population')
    plt.xlabel('Number of Bins')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [6]:
def parent_selection(population):
    selected = random.choices(population, k=5)
    parents = sorted(selected, key=lambda x: x.fitness)
    best1, best2 = parents[:2]
    return best1, best2

In [7]:
# crossover probability
def crossover(parent1, parent2, items_dict, bin_size, pc=1):
    length = len(parent1.value)
    select_offsprings = random.random() < pc
    if not select_offsprings:
        return parent1, parent2

    # Cut
    cut_point = random.randint(0, length - 1)
    parent1_left = parent1.value[:cut_point]
    parent1_right = parent1.value[cut_point:]
    parent2_left = parent2.value[:cut_point]
    parent2_right = parent2.value[cut_point:]

    # Combine genes
    def combine_genes(part1, part2, complement):
        unique_genes = part1 + [x for x in part2 if x not in part1]
        missing_genes = []
        if len(unique_genes) != length:
            missing_genes = [x for x in complement if x not in unique_genes]
        return unique_genes + missing_genes

    offspring1_genes = combine_genes(parent1_left, parent2_right, parent2_left)
    offspring2_genes = combine_genes(parent2_left, parent1_right, parent1_left)

    # Create new Chromosome instances for offspring
    offspring1 = Chromosome(offspring1_genes, items_dict, bin_size)
    offspring2 = Chromosome(offspring2_genes, items_dict, bin_size)
    return offspring1, offspring2

In [8]:
def survival_selection(old_population, new_population, elite_percentage=10):
    total_population_size = len(old_population)
    num_elite = int((elite_percentage / 100.0) * total_population_size)
    combined_population = old_population + new_population
    combined_population.sort(key=lambda x: x.fitness)
    elite = combined_population[:num_elite]
    next_generation_rest = [chrom for chrom in new_population if chrom not in elite]
    next_generation = elite + next_generation_rest[:total_population_size - num_elite]
    return next_generation

In [9]:
import matplotlib.pyplot as plt

def main_loop(instance_name, pop_size, items_dict, bin_size, num_generations):
    gene_pool = list(items_dict.keys())
    population = initialize_population(len(gene_pool), pop_size)
    
    best_fitness_over_time = []
    average_fitness_over_time = []
    best_chromosome = None

    for generation in range(num_generations):
        new_population = []
        if generation % 10 == 0:
            print("generation: ", generation)
        while len(new_population) < pop_size:
            parent1, parent2 = parent_selection(population)
            offspring1, offspring2 = crossover(parent1, parent2, items_dict, bin_size)
            offspring1.mutate()
            offspring2.mutate()
            new_population.extend([offspring1, offspring2])

        population = survival_selection(population, new_population, elite_percentage=10)

        current_fitness_values = [chrom.fitness for chrom in population]
        best_fitness = min(current_fitness_values)
        average_fitness = sum(current_fitness_values) / len(current_fitness_values)

        best_fitness_over_time.append(best_fitness)
        average_fitness_over_time.append(average_fitness)

        # Updating the best chromosome found so far
        if not best_chromosome or best_fitness < best_chromosome.fitness:
            best_chromosome = min(population, key=lambda x: x.fitness)

    if best_chromosome:
        best_chromosome.visualize_bins()
        best_chromosome.write_output("output" + instance_name + ".txt")
        print("best bins count: ", best_chromosome.bin_size)
    return average_fitness_over_time,best_fitness_over_time


In [10]:
instance_names = ['3']
for instance_name in instance_names:
    items_dict, bin_size = read_instance(instance_name)
    num_generations = 200
    pop_size = 100
    average_fitness_over_time,best_fitness_over_time = main_loop(pop_size, items_dict, bin_size, num_generations)
    
    plt.figure(figsize=(10, 5))
    plt.plot(best_fitness_over_time, label='Best Fitness')
    plt.plot(average_fitness_over_time, label='Average Fitness', linestyle='--')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.title('Fitness Trends Over Generations')
    plt.legend()
    plt.grid(True)
    plt.show()

TypeError: main_loop() missing 1 required positional argument: 'num_generations'

In [97]:
instance_names = ['1','2','3','4','5']
for instance_name in instance_names:
    items_dict, bin_size = read_instance(instance_name)
    chromosome = Chromosome.from_file(instance_name, items_dict, bin_size)
    if chromosome:
        print (f"number of bins for instance_{instance_name} is {len(chromosome.bins)}")
        is_feasible = chromosome.check_feasibility()
        print(f"Solution is {'feasible' if is_feasible else 'not feasible'}")
        
    else:
        print("Failed to create Chromosome from file.")

number of bins for instance_1 is 27
Solution is feasible
number of bins for instance_2 is 29
Solution is feasible
number of bins for instance_3 is 457
Solution is feasible
number of bins for instance_4 is 23
Solution is feasible
number of bins for instance_5 is 13
Solution is feasible
